# モジュールのインポート

In [1]:
#coding: shift_jis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure

# CSV読み込み

In [2]:
df = pd.read_csv("c01.csv",encoding="SHIFT-JIS")

# GUIアプリクラス

In [3]:
class GUI_Realtime(tk.Frame):
    def __init__(self, master=None, df=df):
        super().__init__(master)
        self.master = master
        self.master.title('matplotlib')
        self.df = df
        self.df_show = self.df[self.df.都道府県名=="全国"].reset_index(drop=True)
        self.pack()
        self.create_widgets()
        self.start_up()
        self.draw_plot()
    
    def create_widgets(self):
        self.canvas_frame = tk.Frame(self.master)
        self.canvas_frame.pack(side=tk.LEFT)
        self.control_frame = tk.Frame(self.master)
        self.control_frame.pack(side=tk.RIGHT)

        self.canvas = FigureCanvasTkAgg(fig, self.canvas_frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.toolbar = NavigationToolbar2Tk(self.canvas, self.canvas_frame)
        self.toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        self.x_v = tk.DoubleVar()
        self.x_scale = tk.Scale(self.control_frame,
            variable=self.x_v,
            from_=1920.0,
            to=2015.0,
            resolution=5,
            orient=tk.HORIZONTAL,
            command=self.draw_plot)
        self.x_scale.pack(anchor=tk.NW)
        
        # Combobox
        self.v1 = tk.StringVar()
        self.cb = ttk.Combobox(self.control_frame, textvariable=self.v1)
        self.cb.bind('<<ComboboxSelected>>', self.cb_selected)

        self.cb['values']=('全国', '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県')
        self.cb.pack(anchor=tk.NW)

    def start_up(self):
        self.x_v.set(1920.0)
        self.cb.set("全国")
        
    def cb_selected(self, event):
        print(self.v1.get())

    def draw_plot(self, event=None):
        index = int((self.x_v.get() - 1920)/5)
        self.df_show = self.df[self.df.都道府県名==self.cb.get()].reset_index(drop=True)

        ax1.cla()
        self.x = pd.Series(self.df_show["西暦（年）"], dtype=np.float64).reset_index(drop=True)
        self.y = [pd.Series(self.df_show["人口（総数）"], dtype=np.float64).reset_index(drop=True),
                 pd.Series(self.df_show["人口（男）"], dtype=np.float64).reset_index(drop=True),
                 pd.Series(self.df_show["人口（女）"], dtype=np.float64).reset_index(drop=True)]
        self.y_t = [pd.Series(self.df_show["人口（総数）"]).reset_index(drop=True),
                 pd.Series(self.df_show["人口（男）"]).reset_index(drop=True),
                 pd.Series(self.df_show["人口（女）"]).reset_index(drop=True)]

        ax1.set_ylim(0, max(self.y[0])*1.2)
        ax1.set_xlim(min(self.x)*0.999,max(self.x)*1.001)
        
        ax1.plot(self.x[:index+1],self.y[0][:index+1], label="population total", marker="o", color = "blue")
        ax1.plot(self.x[:index+1],self.y[1][:index+1], label="population girl", marker="o", color = "red")
        ax1.plot(self.x[:index+1],self.y[2][:index+1], label="population boy", marker="o", color = "green")

        for x, y, text, y_boy, text_boy, y_girl, text_girl in zip(self.x[:index+1], self.y[0][:index+1], self.y_t[0][:index+1], self.y[1][:index+1], self.y_t[1][:index+1], self.y[2][:index+1], self.y_t[2][:index+1]):
            ax1.text(x-2, y*1.05, '{:.1f}M'.format(float(int(text)/1000000.0)), color = "blue")
            ax1.text(x-2, y_boy*1.05, '{:.1f}M'.format(float(int(text_boy)/1000000.0)), color = "red")
            ax1.text(x-2, y_girl*1.05, '{:.1f}M'.format(float(int(text_girl)/1000000.0)), color = "green")
        self.canvas.draw()

# GUIアプリの起動

In [4]:
fig = Figure(figsize=(10, 10), dpi=100)
ax1 = fig.add_subplot(111)
root = tk.Tk()
app = GUI_Realtime(master=root, df=df)
app.mainloop()